# Fine-tuning Llama 2 in Google Colab
Based on https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32




In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 wandb

In [2]:
import os
import torch
import pandas as pd
import wandb
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# The model that you want to train from the Hugging Face hub
#model_name = "daryl149/llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"

# The instruction dataset to use
#dataset_name = "mlabonne/guanaco-llama2"

# Fine-tuned model name
new_model = "lragnarsson/Llama-2-13b-geocoder"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 8

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 1

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = True

# Load the entire model on the GPU 0
device_map = {"": 0}

# Prepare dataset

In [4]:
wandb.login()

wandb: Currently logged in as: lage. Use `wandb login --relogin` to force relogin


True

In [5]:
%env WANDB_PROJECT=woher

env: WANDB_PROJECT=woher


In [7]:
run = wandb.init(
    project="woher",
    job_type="fine-tuning"
)

artifact = run.use_artifact('lage/woher/llama-cities:latest', type='dataset').get("train")
df = artifact.get_dataframe()

wandb:   1 of 1 files downloaded.  


In [8]:
df

,text
0,<s>[INST] You are a reverse geocoder for city ...
1,<s>[INST] You are a reverse geocoder for city ...
2,<s>[INST] You are a reverse geocoder for city ...
3,<s>[INST] You are a reverse geocoder for city ...
4,<s>[INST] You are a reverse geocoder for city ...
...,...
30618,<s>[INST] You are a reverse geocoder for city ...
30619,<s>[INST] You are a reverse geocoder for city ...
30620,<s>[INST] You are a reverse geocoder for city ...
30621,<s>[INST] You are a reverse geocoder for city ...


In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
tokenizer.eos_token

'</s>'

In [13]:
# Load dataset (you can process it here)
dataset = Dataset.from_pandas(df)
#dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    use_auth_token=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/30623 [00:00<?, ? examples/s]

In [14]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("./saved")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.915200
2,3.787000
3,3.611800
4,3.438600
5,3.242500
6,2.962100
7,2.769700
8,2.578900
9,2.359100
10,2.204200


In [10]:
# Setup Pipeline
#m = model
m = model2
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=m, tokenizer=tokenizer, max_length=200)

In [11]:
PROMPT_TEMPLATE = """<s>[INST]
You are a reverse geocoder for city names.
From a latitude and a longitude, you answer with the most probable city name.
If no real city exists on the location, make up a name that sounds plausible given similar city names in the region.
Extra points for extra stereotypical and clever city names.
Here are the city coordinates: {user_prompt} [/INST] """.replace("\n", " ")


def prompt_model(lat, lon):
  lat = round(lat, 5)
  lon = round(lon, 5)
  input = PROMPT_TEMPLATE.format(user_prompt=f"{lat}, {lon}")
  result = pipe(PROMPT_TEMPLATE.format(user_prompt=input))
  return result[0]['generated_text'].split("[/INST]")[-1]


In [12]:
# Some Test Examples
print("France: ", prompt_model(46.4931465608949, 1.33766730132503))
print("USA: ", prompt_model(41.99311770837206, -103.42473625235849))
print("Africa: ", prompt_model(2.5463442448493567, 21.265506633756175))
print("India: ", prompt_model(21.02933450184807, 78.00560167537144))
print("China: ", prompt_model(26.85780943399129, 114.3951482611622))
print("Sweden: ", prompt_model(58.81358076528444, 15.061218148967365))
print("Germany: ", prompt_model(52.181128595107985, 10.19461552518322))

France:    Saint-Blaise-sur-Richelieu 46.
USA:    Medicine Hat 4.7575, -10
Africa:    Mlimba 
India:    Tung 
China:    Shuangyu 
Sweden:    Bogen 5.06437, 15
Germany:    Neunkirchen 5.4 km Southeast of city


In [13]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

There is a problem with the VRAM here despite `del model` and emptying the VRAM. You probably need to restart the notebook, re-execute the three first cells, and then execute this one. Please contact me if you have a fix!

In [7]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model2 = PeftModel.from_pretrained(base_model, "./saved") # output_dir
model2 = model2.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
model2.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/lragnarsson/Llama-2-13b-geocoder/commit/7df5819f19b3f156dd7424874db07fd7cd961e03', commit_message='Upload tokenizer', commit_description='', oid='7df5819f19b3f156dd7424874db07fd7cd961e03', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
artifact = wandb.Artifact(name='llama-cities', type='dataset')
artifact.add(wandb.Table(dataframe=llama_df), name="train")
wandb.run.log_artifact(artifact)

In [26]:
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▂▂▂▄▄▄▄▄▄▄▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇██████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,0.05
train/global_step,200
